In [1]:
import openpyxl

workbook = openpyxl.Workbook()

# 保存工作簿
workbook.save('ds.xlsx')

In [1]:
from web3 import Web3, EthereumTesterProvider
from web3.middleware import geth_poa_middleware
import web3
import solcx
import json
import time
import re
from solcx import install_solc
import random
import threading
import time
from web3.contract import Contract
from datetime import datetime
import openpyxl


# create Web3 object connecting to the node(Geth) of  execution layer
switch1_node_addr=[] # 交换机1下节点的地址列表
switch1_node_list=[] # 交换机1下节点的Web3对象列表
for i in range(47):
    switch1_node_addr.append("http://192.168.1."+str(i+1)+":"+str(6000+i+1))
    switch1_node_list.append(Web3(web3.HTTPProvider(switch1_node_addr[i])))
switch2_node_addr=[]
switch2_node_list=[]
for i in range(16):
    switch2_node_addr.append("http://192.168.2."+str(i+1)+":"+str(6000+i+1+47))
    switch2_node_list.append(Web3(web3.HTTPProvider(switch2_node_addr[i])))
switch3_node_addr=[]
switch3_node_list=[]
for i in range(1):
    switch3_node_addr.append("http://192.168.3."+str(i+1)+":"+str(6000+i+1+63))
    switch3_node_list.append(Web3(web3.HTTPProvider(switch3_node_addr[i])))
total_node_list=[]
total_node_list.extend(switch1_node_list)
total_node_list.extend(switch2_node_list)
total_node_list.extend(switch3_node_list)
for i in total_node_list:
    print(i.is_connected(),end=' ')
# node1_ipaddr = "http://192.168.1.1:6001"
# node1 = Web3(Web3.HTTPProvider(node1_ipaddr))
# node2_ipaddr = "http://192.168.1.1:6002"
# node2 = Web3(Web3.HTTPProvider(node2_ipaddr))
# node3_ipaddr = "http://192.168.1.1:6003"
# node3 = Web3(Web3.HTTPProvider(node3_ipaddr))
# node4_ipaddr = "http://192.168.1.1:6004"
# node4 = Web3(Web3.HTTPProvider(node4_ipaddr))
# print(
#     node1.is_connected(),
#     node2.is_connected(),
#     node3.is_connected(),
#     node4.is_connected(),
# )
print()

# 解锁账户使之能部署合约
account_password = "1"  # 设置新账户的密码
# 解锁交换机1下的账户
def unlock_account_task(node_list:list):
    for node in node_list:
        unlock=node.geth.personal.unlock_account(node.eth.accounts[0],account_password,0)
        print(unlock,end=' ')
    print(f'{len(node_list)} node account unlock completed')

unlock_thread1=threading.Thread(target=unlock_account_task,args=(total_node_list[0:12],))
unlock_thread1.start()
unlock_thread2=threading.Thread(target=unlock_account_task,args=(total_node_list[12:24],))
unlock_thread2.start()
unlock_thread3=threading.Thread(target=unlock_account_task,args=(total_node_list[24:36],))
unlock_thread3.start()
unlock_thread4=threading.Thread(target=unlock_account_task,args=(total_node_list[36:48],))
unlock_thread4.start()
unlock_thread5=threading.Thread(target=unlock_account_task,args=(total_node_list[48:64],))
unlock_thread5.start()

True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True 


True True True True True True True True True True True True True True True True True True TrueTrue  True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True 12 node account unlock completed
True True 12 node account unlock completed
True 12 node account unlock completed
True 12 node account unlock completed
True True True True 16 node account unlock completed


In [2]:
# deploy contract: input the node for deploying contract
def deployContract(node: Web3):
    with open("erc20_ds.solidi") as f:
        contract_src_code = f.read()
        compiled_sol = solcx.compile_source(
        contract_src_code, output_values=["abi", "bin"], solc_version="0.8.19"
    )
    contract_id, contract_interface = compiled_sol.popitem()
    bytecode = contract_interface["bin"]
    abi = contract_interface["abi"]
    node.eth.default_account = node.eth.accounts[0]
    MyContract = node.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = MyContract.constructor().transact()
    tx_receipt = node.eth.wait_for_transaction_receipt(tx_hash)
    print("Contract deployed at address: ", tx_receipt.contractAddress)
    return tx_receipt.contractAddress, abi


# deploy contract and send some money to the four nodes in the network
total_contract_list=[]
contract_address, abi = deployContract(total_node_list[0])
for node in total_node_list:
    node.eth.default_account=node.eth.accounts[0]
    contract=node.eth.contract(address=contract_address,abi=abi)
    total_contract_list.append(contract)
    # contract.functions.setBalance(node.eth.default_account).transact()



Contract deployed at address:  0x3e2AabB763F255CbB6a322DBe532192e120B5C6B


In [3]:
# 对主网和子网分别发送100笔交易 观察两边的计数器大小
for i in range(100):
    total_contract_list[0].functions.decrementByThree().transact()# 主网操作
    total_contract_list[57].functions.decrementByFour().transact()# 子网操作

while(1):
    print(f'主网计数器:{total_contract_list[0].functions.getCounter().call()}  子网计数器:{total_contract_list[57].functions.getCounter().call()}')
    time.sleep(2)

主网计数器:9757  子网计数器:10000
主网计数器:9757  子网计数器:9600
主网计数器:9757  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9600
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700  子网计数器:9588
主网计数器:9700

ConnectionError: HTTPConnectionPool(host='192.168.1.1', port=6001): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f37676c3850>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [28]:
"""
统计某个节点返回来的交易数量交易
"""


def get_TransactionNumFromNode(node: Web3) -> int:
    currentBlockNum = node.eth.get_block_number()
    tx_num = 0
    for i in range(currentBlockNum):
        currentBlockTxNum = node.eth.get_block_transaction_count(i)
        tx_num+=currentBlockTxNum
    return tx_num

"""
根据到达率获取泊松分布的到达时间用于交易发送的间隔
"""
arrivate_rate = 100


def get_interarrival_time():
    return random.expovariate(arrivate_rate)


"""
吞吐量测试线程中的任务一：节点持续调用智能合约转账
"""
def tx_task(contract: Contract, sender: Web3, receiver: Web3):
    while True:
        interval = get_interarrival_time()
        time.sleep(interval)
        contract.functions.sendCoin(
            sender.eth.accounts[0], receiver.eth.accounts[0], 1
        ).transact()


"""
吞吐量测试线程中的任务二：输入起始的交易数量txNum_start 输出主网络节点node经过了几秒交易数增加了多少
"""
def tps_mainnet_task(node: Web3, txNum_start: int, start_time: datetime):
    workbook = openpyxl.load_workbook("/usr/local/myproj/ethereumLab/testCode/tps_mainnet.xlsx")
    ws = workbook.create_sheet("512kb", 0)
    i = 0
    while i < 5:
        current_time = datetime.now()
        txNum_temp = get_TransactionNumFromNode(node=node)  # 当前上链交易总数
        ws.append(
            (
                start_time.timestamp(),
                current_time.timestamp(),
                (current_time - start_time).total_seconds(),
                txNum_temp - txNum_start,
            )
        )
        print(
            "在节点%s中，从%f秒到%f秒，%f秒内上链交易数增加了%d"
            % (
                node.eth.accounts[0],
                start_time.timestamp(),
                current_time.timestamp(),
                (current_time - start_time).total_seconds(),
                txNum_temp - txNum_start,
            )
        )
        txNum_start = txNum_temp
        start_time = current_time
        i += 1
        time.sleep(60)
    workbook.save("tps_mainnet.xlsx")
    print("tps measure complete for mainnet")
    workbook.close()

"""
吞吐量测试线程中的任务三：输入起始的交易数量txNum_start 输出子网络节点node经过了几秒交易数增加了多少
"""
def tps_subnet_task(node: Web3, txNum_start: int, start_time: datetime):
    workbook = openpyxl.load_workbook("/usr/local/myproj/ethereumLab/testCode/tps_subnet.xlsx")
    ws = workbook.create_sheet("512kb", 0)
    i = 0
    while i < 5:
        current_time = datetime.now()
        txNum_temp = get_TransactionNumFromNode(node=node)  # 当前上链交易总数
        ws.append(
            (
                start_time.timestamp(),
                current_time.timestamp(),
                (current_time - start_time).total_seconds(),
                txNum_temp - txNum_start,
            )
        )
        print(
            "在节点%s中，从%f秒到%f秒，%f秒内上链交易数增加了%d"
            % (
                node.eth.accounts[0],
                start_time.timestamp(),
                current_time.timestamp(),
                (current_time - start_time).total_seconds(),
                txNum_temp - txNum_start,
            )
        )
        txNum_start = txNum_temp
        start_time = current_time
        i += 1
        time.sleep(60)
    workbook.save("tps_subnet.xlsx")
    print("tps measure complete for subnet")
    workbook.close()

"""
吞吐量测试线程中的任务四：输入节点对象 输出前blockNum之前区块的测量区块的出块时延
"""
def block_Delay_task(node: Web3,nodeNum:int):

    total_block_number=node.eth.block_number
    # 从第三个区块开始记录
    delayTimeList=[]
    current_block_number=3
    preTimeStamp=node.eth.get_block(current_block_number-1).timestamp
    while current_block_number<total_block_number:
        current_block_timestamp=node.eth.get_block(current_block_number).timestamp
        delayTimeList.append(current_block_timestamp-preTimeStamp)
        preTimeStamp=current_block_timestamp
        current_block_number+=1
    average_delay=sum(delayTimeList)/len(delayTimeList)

    print(f"In node {nodeNum} Average block time delay over the last {total_block_number-3} blocks: {average_delay} seconds")
    # print(f"Total number of transactions in the last {block_count} blocks: {total_transactions}")
    # print(f"Blockchain throughput: {throughput} transactions per second")
    print("block delay task complete")


In [23]:
"""
测试函数
"""


def tpsCal():
    thread1 = threading.Thread(
        target=tx_task, args=(total_contract_list[0], total_node_list[0], total_node_list[57])
    )  #
    thread1.start()
    mainnet_txNum_start = get_TransactionNumFromNode(total_node_list[0])
    subnet_txNum_start=get_TransactionNumFromNode(total_node_list[57])
    thread2 = threading.Thread(
        target=tps_mainnet_task, args=(total_node_list[0], mainnet_txNum_start, datetime.now())
    )  # 打印节点1的输出
    thread2.start()
    thread3=threading.Thread(
        target=tps_subnet_task, args=(total_node_list[57], subnet_txNum_start, datetime.now())
    )
    thread3.start()
    thread2.join()
    thread3.join()
    
    thread4=threading.Thread(target=block_Delay_task,args=(total_node_list[0],0,))
    thread4.start()
    thread5=threading.Thread(target=block_Delay_task,args=(total_node_list[57],57,))
    thread5.start()

In [24]:
tpsCal()

在节点0x123463a4B065722E99115D6c222f267d9cABb524中，从1703055429.268501秒到1703055429.287380秒，0.018879秒内上链交易数增加了0
在节点0x72D44a97894639D5B7AA86911f0139e418Bf3149中，从1703055429.268667秒到1703055429.295715秒，0.027048秒内上链交易数增加了0
在节点0x72D44a97894639D5B7AA86911f0139e418Bf3149中，从1703055429.295715秒到1703055489.437727秒，60.142012秒内上链交易数增加了161
在节点0x123463a4B065722E99115D6c222f267d9cABb524中，从1703055429.287380秒到1703055489.433773秒，60.146393秒内上链交易数增加了1396
在节点0x72D44a97894639D5B7AA86911f0139e418Bf3149中，从1703055489.437727秒到1703055549.601530秒，60.163803秒内上链交易数增加了134
在节点0x123463a4B065722E99115D6c222f267d9cABb524中，从1703055489.433773秒到1703055549.631835秒，60.198062秒内上链交易数增加了1638
在节点0x72D44a97894639D5B7AA86911f0139e418Bf3149中，从1703055549.601530秒到1703055609.748978秒，60.147448秒内上链交易数增加了0
在节点0x123463a4B065722E99115D6c222f267d9cABb524中，从1703055549.631835秒到1703055609.849540秒，60.217705秒内上链交易数增加了1693
在节点0x72D44a97894639D5B7AA86911f0139e418Bf3149中，从1703055609.748978秒到1703055669.881531秒，60.132553秒内上链交易数增加了0
在节点0x123463a4B065722E99115

In node 57 Average block time delay over the last 48 blocks: 7.25 seconds
block delay task complete
In node 0 Average block time delay over the last 94 blocks: 3.797872340425532 seconds
block delay task complete


Exception in thread Thread-671:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 665, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.8/http/client.py", line 1256, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.8/http/cl

In [45]:
total_node_list[0].eth.get_block(100).timestamp

BlockNotFound: Block with id: '0x64' not found.

Exception in thread Thread-385:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 665, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.8/http/client.py", line 1256, in request
    self._send_request(method, url, body, headers, encode_chunked)
  Fil